In [1]:
import numpy as np
import pandas as pd
import os
# Récupérer le fichier parent en tant que BASE_DIR
BASE_DIR = "C:/Users/mangg/projects/rakutenteam"
os.chdir(BASE_DIR)

In [2]:
df = pd.read_csv("data/clean/X_train.csv", index_col=0)
df.head()

,designation,description,productid,imageid,language,designation_translated,description_translated
0,Olivia: Personalisiertes Notizbuch / 150 Seite...,NaN,3804725264,1263597046,de,Olivia : Carnet personnalisé / 150 pages / gri...,NaN
1,Journal Des Arts (Le) N° 133 Du 28/09/2001 - L...,NaN,436067568,1008141237,fr,Journal Des Arts (Le) N° 133 Du 28/09/2001 - L...,NaN
2,Grand Stylet Ergonomique Bleu Gamepad Nintendo...,PILOT STYLE Touch Pen de marque Speedlink est ...,201115110,938777978,fr,Grand Stylet Ergonomique Bleu Gamepad Nintendo...,PILOT STYLE Touch Pen de marque Speedlink est ...
3,Peluche Donald - Europe - Disneyland 2000 (Mar...,NaN,50418756,457047496,en,Peluche Donald - Europe - Disneyland 2000 (Mar...,NaN
4,La Guerre Des Tuques,Luc a des idées de grandeur. Il veut organiser...,278535884,1077757786,fr,La Guerre Des Tuques,Luc a des idées de grandeur. Il veut organiser...


In [3]:
df["full_text"] = df["designation_translated"] + " " + df["description_translated"].fillna("")

In [4]:
df.full_text.head()

0    Olivia : Carnet personnalisé / 150 pages / gri...
1    Journal Des Arts (Le) N° 133 Du 28/09/2001 - L...
2    Grand Stylet Ergonomique Bleu Gamepad Nintendo...
3    Peluche Donald - Europe - Disneyland 2000 (Mar...
4    La Guerre Des Tuques Luc a des idées de grande...
Name: full_text, dtype: object

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [6]:
vectorizer = TfidfVectorizer(max_features=1000)
vectorizer.fit(df.full_text)

TfidfVectorizer(max_features=1000)

In [7]:
vectorized_text = vectorizer.transform(df.full_text)
vectorized_text.shape

(84916, 1000)

In [8]:
selected_words = vectorizer.get_feature_names_out()

In [9]:
df_word_vecs = pd.DataFrame(vectorized_text.toarray(), columns=selected_words)

In [10]:
new_df = pd.concat([df, df_word_vecs], axis=1)
new_df.shape

(84916, 1008)

In [11]:
new_df.columns

Index(['designation', 'description', 'productid', 'imageid', 'language',
       'designation_translated', 'description_translated', 'full_text', '01',
       '04',
       ...
       'éléments', 'énergie', 'épaisseur', 'éponge', 'équipement', 'étanche',
       'état', 'été', 'éviter', 'être'],
      dtype='object', length=1008)

In [12]:
data = new_df.drop(columns=["designation", "description", "full_text", "productid", "imageid", "designation_translated", "description_translated", "language"])

In [13]:
data.head()

,01,04,05,10,100,11,12,120,122,13,...,éléments,énergie,épaisseur,éponge,équipement,étanche,état,été,éviter,être
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.206854


In [14]:
target = pd.read_csv("data/clean/Y_train.csv", index_col=0)["prdtypecode"]
target.shape

(84916,)

In [15]:
target.head()

0      10
1    2280
2      50
3    1280
4    2705
Name: prdtypecode, dtype: int64

In [16]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC

In [17]:
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2, random_state=42)

In [32]:
clf = SVC()
clf.fit(X_train, y_train.values)

SVC()

In [33]:
clf.score(X_test, y_test)

0.7212081959491285

In [34]:
import numpy as np

In [35]:
param_grid = {
    "C": [np.logspace(-4, 2, 10)],
    "kernel": ["linear", "poly", "rbf", "sigmoid", "precomputed"],
    "degree": range(6)
}

In [36]:
clf = SVC()

In [37]:
grid = GridSearchCV(estimator=clf, param_grid=param_grid, cv=5, n_jobs=-1)

In [ ]:
grid.fit(X_train, y_train.values)

In [ ]:
results = pd.DataFrame(gridsearch.cv_results_)

In [ ]:
final_clf = gridsearch.best_estimator_

In [ ]:
final_clf.score(X_test, y_test)

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
y_pred = final_clf.predict(X_test)

In [ ]:
print(classification_report(y_test, y_pred))

In [ ]:
pd.crosstab(y_test, y_pred, rownames=["Classes Réelles"], colnames=["Prédictions"], margins=True)